In [1]:
#init

import numpy as np   #numpy lets us do nice vector manipulations
import glob          #access the file system (for searching for files)
%matplotlib widget
import matplotlib.pyplot as plt          #for plotting
from scipy.signal import savgol_filter   #nice filter
#import mpld3                             #toolbar for zooming in plots
#mpld3.enable_notebook()

In [2]:
#import data


#Load data (all .ASC files in directory)
files=sorted(glob.glob("*.ASC"))

#Get x axis wavenumbers from a file
x = np.loadtxt(files[1], skiprows=25, unpack=True)[0]

n=len(files)            #number of files
m=len(x)             #data length of each spectrum
data=np.ndarray((n,m))  #create nxm array to hold all data

for i in range(n):
    data[i] = np.loadtxt(files[i], skiprows=25, unpack=True)[1]

In [3]:
#Smooth data

sdata=np.ndarray((n,m))   #another array to hold smoothed data

for i in range(n):
    sdata[i]=savgol_filter(data[i],23,3)

In [4]:
#Find peaks (in smoothed data)

peaks=[]

for i in sdata:
    peaks.append(np.argmax(i[1700:1800])+1700)    #find peak

In [5]:
#Normalize

nsdata=np.ndarray((n,m))

for i in range(n):
    tmp=sdata[i]-min(sdata[i])   #subtract minimum
    #sum=np.sum(data[i])
    #data[i]/=sum   #divide by sum (area normalization)
    nsdata[i]=tmp/tmp[peaks[i]]
    
x[peaks]

array([2552., 2548., 2550., 2544., 2558., 2554., 2546., 2414., 2406.,
       2408., 2406.])

In [6]:
fig, ax = plt.subplots()
ax.set_title('Uncoupled ODH stretch, eHDA 1% HOD in D2O, spectra at 80K after heating')

#fig=plt.figure(figsize=(15,10))   #creates a figure and sets size

#include=[0,1,2,3,4,7,8,9]       #These spectra will be plotted

legend=[str(files[i])+' (' + str(x[peaks[i]])+')' for i in range(n)]   #Use filenames for legend   
#legend=['80 K', '10000 K', 'etc']   #custom legend

lines=[0 for i in range(n)]

for i in range(n):
   lines[i], = ax.plot(x,sdata[i]-sdata[i][peaks[i]], linestyle='-')

leg = ax.legend(legend, loc='upper right')
leg.get_frame().set_alpha(0.5)

plt.ylabel('Absorbance [arb. u]')
plt.xlabel('Wavenumber [per cm]')

# we will set up a dict mapping legend line to orig line, and enable
# picking on the legend line
#lines = [line1, line2]
lined = dict()
for legline, origline in zip(leg.get_lines(), lines):
    legline.set_picker(5)  # 5 pts tolerance
    lined[legline] = origline

def onpick(event):
    # on the pick event, find the orig line corresponding to the
    # legend proxy line, and toggle the visibility
    legline = event.artist
    origline = lined[legline]
    vis = not origline.get_visible()
    origline.set_visible(vis)
    # Change the alpha on the line in the legend so we can see what lines
    # have been toggled
    if vis:
        legline.set_alpha(1.0)
    else:
        legline.set_alpha(0.2)
    fig.canvas.draw()

fig.canvas.mpl_connect('pick_event', onpick)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
temps=[80,90,100,105,110,115,120,125,130,140,150,160]
plt.plot(temps,x[peaks])
plt.title('OD peak position at 80K after heating')
plt.ylabel('Wavenumber [per cm]')
plt.xlabel('Heating temperatur [K]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …